In [1]:
from common import *
from matrix import *
from helper.weighted import *
%matplotlib inline
canvas(16, 12)
import pickle

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [5]:
from models.thealg import alg

In [6]:
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    days = pd.Series([int(x) for x in file.read().splitlines()], name='Day')
n_days = len(days)
days_a, days_b, days_c = days[:n_days // 3], days[n_days // 3:(2 * n_days) // 3], days[(2 * n_days) // 3:]
#days_a, days_b = set(days[:n_days // 2]), set(days[n_days // 2:])
tr, cv, ep = df.Day.isin(days_a), df.Day.isin(days_b), df.Day.isin(days_c)
#tr, cv = df.Day.isin(days_a), df.Day.isin(days_b)

In [ ]:
raise Stop

In [7]:
duse = dfh # <=============
rawfeats = x_feats # <============
def_matrices(globals(), duse, 10000)
for xname in rawfeats:
    X = duse.pivot('Day', 'Stock', xname)
    Xa = X.fillna(0)
    Xm = X.fillna(X.mean())
    Xpct = X.pct()
    Xpcta = Xpct.fillna(0)
    Xpctm = Xpct.fillna(0.5)
    for ext in ['', 'a', 'm', 'pct', 'pcta', 'pctm']:
        globals()[xname.upper() + ext] = locals()['X' + ext]

In [8]:
def mse(Y_, st):
    return (W[st] * (Y_ - Y[st]) ** 2).sum().sum() / W[st].sum().sum()
def rmse(Y_, st):
    return np.sqrt(mse(Y_, st))
def prss(Y_, st):
    return mse(Y_, st) / mse(0, st)
def lbss(Y_, st):
    return prss(Y_, st) * 0.500944
def metric(Y_, st):
    print('(' + ', '.join('{:.6f}'.format(round(func(Y_, st), 6)) for func in [rmse, prss, lbss]) + ')')

In [9]:
Xa = pd.concat({x: globals()[x.upper() + 'a'] for x in rawfeats}, axis=1)#.swaplevel(axis=1).sort_index(axis=1)
Xpcta = pd.concat({x: globals()[x.upper() + 'pcta'] for x in rawfeats}, axis=1)#.swaplevel(axis=1).sort_index(axis=1)

In [10]:
Xuse = Xpcta #.loc[:, (ss, 'x4')]

### testing! / easy

In [5]:
Ya = make_stock_wide(dh.y).fillna(0)
Wa = make_stock_wide(dh.Weight).fillna(0)

In [6]:
_demean = lambda x: x - x.wmean(Wa)

In [9]:
X = dh[x_feats].pipe(makesw)
Xoo = X.replace(np.nan, np.inf)

#### features

In [27]:
xfd = dh[['Day']]
xfdw = ((xfd + 2) % 7).colname_append('.wkd')
_xfdq = []
for w in range(13):
    _x = (xfd + 2 + w * 7) % 91
    _xfdq.append(_x.colname_append('.qtr({})'.format(w)))
xfdq = pd.concat(_xfdq, axis=1)
Xfdt = pd.concat([xfd, xfdw, xfdq], axis=1).pipe(makesw).pipe(_demean)

In [30]:
save_dir = big_dir + '/data/features/'
fnames = 'x_.mmsgnco x_.log.mmsgnco x3.mmsgnco x3.log.mmsgnco'.split()
raw_, pct_ = [1, 1, 1, 1], [1] * 4
_Xfsi = []
for fname, r_, p_ in zip(fnames, raw_, pct_):
    print(fname, end='  ')
    _Xo = pd.read_hdf(save_dir + '64/' + fname + '.hdf5')
    print('check 1', end='  ')
    if r_:
        _X = _Xo.pipe(makesw).pipe(_demean)
        _Xfsi.append(_X.astype(np.float32, copy=False))
    print('check 2', end='  ')
    if p_:
        _X = _Xo.colname_append('.wpct').pipe(makesw).wpct(Wa, [-1, 1])
        _Xfsi.append(_X.astype(np.float32, copy=False))
    print('check 3')
    del _X
    print()
Xfsi = pd.concat(_Xfsi, axis=1)
del _Xfsi
gc.collect()

x_.mmsgnco  check 1  check 2  

/home/ec2-user/Notebooks/g-research-ffc2018/helper/weighted.ipynb:11: RuntimeWarning: invalid value encountered in true_divide
  "from common import *"


check 3

x_.log.mmsgnco  check 1  check 2  check 3

x3.mmsgnco  check 1  check 2  check 3

x3.log.mmsgnco  check 1  check 2  check 3



70

In [31]:
Xa = pd.concat([Xfdt, Xfsi], axis=1).fillna(0)

In [32]:
sfilt = stock_nvdf(dh, 246) # <==============
Xtest = Xa.iloc[:, np.concatenate([sfilt] * Xa.columns.levels[0].shape[0])]
Ytest = Ya.T[sfilt].T
Wtest = Wa.T[sfilt].T
tr, cv, ep = Ytest.index.isin(days_a), Ytest.index.isin(days_b), Ytest.index.isin(days_c)
scores_, models_ = alg(Xtest[tr], Ytest[tr], Wtest[tr],
                     Xtest[cv], Ytest[cv], Wtest[cv],
                     Xtest[ep], Ytest[ep], Wtest[ep], intercept=False)

0.7520035710362881 0.762786104189627 0
()()

[4871]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........500.........510.........520.........530.........540.........550.........560.........570.........580.........590.........600.........610.........620.........630.........640.........650.........660.........670.........680.........690.........700.........710.........720.........730.........740.........750.........760.........770.........780.........790.........

[4869]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........500.........510.........520.........530.........540.........550.........560.........570.........580.........590.........600.........610.........620.........630.........640.........650.........660.........670.........680.........690.........700.........710.........720.........730.........740.........750.........760.........770.........780.........790.........800.........810.........820.........830......

[4867]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........500.........510.........520.........530.........540.........550.........560.........570.........580.........590.........600.........610.........620.........630.........640.........650.........660.........670.........680.........690.........700.........710.........720.........730.........740.........750.........760.........770.........780.........790.........800.........810.........820.........830......

In [82]:
import pickle
k = Xtest.columns.get_level_values(0).nunique() # features
m = Xtest.columns.get_level_values(1).nunique() # stocks/groups
feats = [str(Xtest.columns[m * f][0]) for f in range(k)]

In [3]:
pname = '[247]mmsgnco__first'

In [97]:
with open(top_dir + 'data/progress/linsearch/' + pname + '.pkl', 'wb') as file:
    pickle.dump((feats, scores), file, protocol=-1)

In [6]:
with open(top_dir + 'data/progress/linsearch/' + pname + '.pkl', 'rb') as file:
    feats, scores = pickle.load(file)

In [7]:
see = {tuple(feats[j] for j in k): s for k, s in scores.items()}

In [8]:
baser = 0.7520035710362881, 0.762786104189627
see[None] = see[()][0] / baser[0], see[()][1] / baser[1]

In [9]:
bl = baseline = see[None]
baseline

(16004879.657038068, 15750835.079447325)

In [12]:
sub = [((s[0] + s[1]) / (bl[0] + bl[1]), k[0]) for k, s in see.items()
       if k is not None and len(k) == 1 and 'wpct' in k[0]]
[print(line) for line in sorted(sub)]

(0.658999091185371, 'x_.log.sgnint(03/45).wpct')
(0.6602654752434745, 'x_.log.sgnint(03/456).wpct')
(0.6603426637284927, 'x3.log.sgnint(01/34).wpct')
(0.6611277772613557, 'x3.log.sgnint(02/34).wpct')
(0.6640730821413394, 'x3.log.sgnint(0/4).wpct')
(0.6641792420953331, 'x3.log.sgnint(01/24).wpct')
(0.6652151340756118, 'x_.log.sgnint(036/45).wpct')
(0.6671481799913757, 'x_.log.sgnint(03/4).wpct')
(0.6673226980224106, 'x_.log.sgnint(013/456).wpct')
(0.6674356257155478, 'x_.log.sgnint(013/45).wpct')
(0.6677032389993056, 'x_.log.sgnint(03/46).wpct')
(0.6694655936952211, 'x_.log.sgnint(03/5).wpct')
(0.6694738557808149, 'x_.log.sgnint(023/45).wpct')
(0.6695467813402082, 'x_.log.sgnint(023/456).wpct')
(0.6696218193869236, 'x_.log.sgnint(0/4).wpct')
(0.6702529826424682, 'x_.log.sgnint(03/245).wpct')
(0.6705160437508796, 'x_.log.sgnint(03/56).wpct')
(0.671744974926242, 'x_.log.sgnint(03/145).wpct')
(0.6719966799674075, 'x3.log.sgnint(12/34).wpct')
(0.6723818234640295, 'x_.log.sgnint(03/2456).wpc

(0.7544526989621609, 'x_.sgnint(0/14).wpct')
(0.7544529907342246, 'x_.sgnint(03/1).wpct')
(0.7544532794260723, 'x_.sgnint(0/13).wpct')
(0.7544534406766326, 'x_.sgnint(03/145).wpct')
(0.7544539382618203, 'x_.sgnint(05/134).wpct')
(0.7544542767101257, 'x_.sgnint(0/134).wpct')
(0.7544550890798098, 'x_.sgnint(0345/1).wpct')
(0.7544556934668337, 'x_.sgnint(045/1).wpct')
(0.7544558362153009, 'x_.sgnint(034/15).wpct')
(0.7544561420763166, 'x_.sgnint(034/1).wpct')
(0.7544561785859277, 'x_.sgnint(04/15).wpct')
(0.7544562056296094, 'x_.sgnint(04/1).wpct')
(0.7544580543420165, 'x_.sgnint(045/13).wpct')
(0.7544583266924269, 'x_.sgnint(04/135).wpct')
(0.7544584324305986, 'x_.sgnint(04/13).wpct')
(0.7546736647813189, 'x_.sgnint(012345/).wpct')
(0.7546740798905465, 'x_.sgnint(01234/5).wpct')
(0.7546744681896236, 'x_.sgnint(01234/).wpct')
(0.7546753185605825, 'x_.sgnint(01245/).wpct')
(0.7546756939389815, 'x_.sgnint(0124/).wpct')
(0.7546766287399365, 'x_.sgnint(0124/5).wpct')
(0.754677327406245, 'x_.s

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

#### small

In [ ]:
from sklearn.linear_model import LinearRegression as LR

def do_lr(X, Y, W):
    lr_list = []
    # assert the columns of the input can be indexed by range(0, 3023)
    for k in range(N_STOCK):
        x, y, w = X[k], Y[k], W[k]
        if w.sum() == 0:
            lr_list += [None]
            continue
        lr = LR(fit_intercept=True)
        # don't forget x1 and x2 have NaNs for valid y. try setting W=0 for when y and W does not exist and then filling in NaNs
        lr = lr.fit(x, y, w)
        lr_list += [lr]
    return lr_list

In [66]:
lr_list = do_lr(Xuse[tr], Ya[tr], Wa[tr])

In [51]:
Yx = np.stack([(lr.predict(Xuse[k]) if lr is not None else np.zeros(Y.shape[0]))
               for lr, k in zip(lr_list, range(N_STOCK))], axis=1)
Yx = Yx + Y * 0

In [52]:
st = cv
Yxm = Yx.where(Y[tr].count() > 50, Y[tr].mean().fillna(0), axis=1)

#### big

In [20]:
bigX = Xpcta.stack(level=0)
bigY = Ya.stack(level=0)
bigW = Wa.stack(level=0)

In [24]:
lr = LR()
lr = lr.fit(bigX, bigY, bigW)

In [28]:
Yx = np.stack([(lr.predict(Xpcta[k]) if lr is not None else np.zeros(Y.shape[0]))
               for k in range(N_STOCK)], axis=1)
Yxm = Yx + Y * 0

### metric test sees

In [53]:
metric(Yxm[tr], tr)

(2.869601, 0.748053, 0.374732)


In [54]:
metric(Yxm[cv], cv)

(2.927475, 0.795920, 0.398711)


In [21]:
metric(Y[tr].wmean(W[tr]), cv)

(2.951820, 0.809212, 0.405370)


In [22]:
metric(Y[tr].mean(), cv)

(2.961588, 0.814577, 0.408057)


In [19]:
metric(0, cv)

(3.281396, 1.000000, 0.500944)


In [18]:
metric(Y[cv]*.225, cv)

(2.543082, 0.600625, 0.300879)


### stop

In [ ]:
raise Stop

In [ ]:
k = 2
plt.scatter(Xuse[k], Ya[k]);
plt.plot(Xuse[k], Yx[k]);
plt.xlim(-0.0001, 0.0004);
plt.ylim(-0.0002, 0.0001);